## Instalación de dependencias

In [ ]:
# Install core libraries for text modeling and evaluation
%pip -q install -U datasets accelerate transformers peft trl bitsandbytes sentencepiece einops
%pip -q install -U matplotlib seaborn sentence_transformers

# helpers
%pip install wordcloud

# Install Accelerate for efficient training and device management
%pip install "accelerate>=0.26.0"

## Configuración del entorno de ejecución (GPU y Accelerate)

Establecemos variables de entorno para controlar el uso de GPU y evitar problemas comunes al entrenar modelos en notebooks. En particular, se fuerza el uso de una única GPU y se desactiva el mixed precision de Accelerate para mejorar la estabilidad del entrenamiento en este entorno.

In [ ]:
import os

# Use a single GPU to avoid DataParallel issues in notebooks.
os.environ["CUDA_VISIBLE_DEVICES"] = "0" # Or choose any GPU free

# Force Accelerate to disable mixed precision (stability fix for this environment).
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"

## Importación de librerías

Esta sección carga todas las dependencias necesarias para el procesamiento de texto, entrenamiento del modelo y utilidades auxiliares.

In [ ]:
import csv
import gc
import math
import random
import re
from collections import Counter
from datetime import datetime, timedelta
from pathlib import Path

import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
from datasets import DatasetDict, load_dataset
from IPython.display import clear_output, display
from peft import (
    LoraConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)
from trl import SFTConfig, SFTTrainer

## Configuración del dispositivo de ejecución

Establecemos el dispositivo de cómputo a utilizar durante el entrenamiento y la inferencia. Utilizamos la GPU si está disponible, de lo contrario el código se ejecuta en CPU.

In [ ]:
# Detect compute device (informativo)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Dispositivo seleccionado: {device}")

if torch.cuda.is_available():
    print(f"GPUs disponibles: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        name = torch.cuda.get_device_name(i)
        cap = torch.cuda.get_device_capability(i)
        print(f"  GPU {i}: {name} — Compute Capability: {cap}")

## Parámetros y filtros de datos irrelevantes

En esta sección se definen patrones típicos de WhatsApp que deben ser descartados durante la limpieza del dataset.

In [ ]:
# Set of patterns considered irrelevant for training (WhatsApp system messages)
irrelevant_data = {
    # Spanish
    "eliminaste este mensaje",
    "se eliminó este mensaje",
    "<multimedia omitido>",
    "multimedia omitido",
    "los mensajes y las llamadas están cifrados de extremo a extremo",

    # English
    "you deleted this message",
    "this message was deleted",
    "<media omitted>",
    "media omitted",
    "messages and calls are end-to-end encrypted",
}

def contains_irrelevant_data(message: str) -> bool:
    """
    Returns True if the message contains any irrelevant WhatsApp system string.
    Assumes the input message has already been lowercased.
    """
    return any(pattern in message for pattern in irrelevant_data)

## Procesamiento del chat de WhatsApp

Esta celda contiene todas las funciones relacionadas con la limpieza, parseo y estructuración del chat de WhatsApp.
No realiza acciones por sí misma, solo define el procesamiento que luego será utilizado por la interfaz interactiva.
Contiene distintos separadores:

* Tags con los que el modelo fue entrenado: <|system|>, <|user|>, <|assistant|>, <|end|>
* Tags utilizados para los diferenciar interlocutores en los turnos [_Autor_], [OTRO]

In [ ]:
MSG_SEP = "<|msg_sep|>"  # Separator used inside a single turn


def clean_text(text: str) -> str:
    """Light text normalization for WhatsApp messages."""
    text = text.lower().strip()
    text = re.sub(r"[^a-záéíóúñü0-9,.;:¡!¿?\s']", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def parse_datetime(line: str):
    """Parse WhatsApp timestamp from a message line, if present."""
    match = re.match(r"(\d+/\d+/\d+[, ]\s?\d+:\d+)\s-", line)
    if not match:
        return None

    raw = match.group(1).replace(",", "")
    for fmt in ("%d/%m/%y %H:%M", "%d/%m/%Y %H:%M"):
        try:
            return datetime.strptime(raw, fmt)
        except ValueError:
            continue
    return None


def group_consecutive_messages(messages):
    """Group consecutive messages from the same author when close in time."""
    grouped = []
    for author, msg, ts in messages:
        if (
            grouped
            and grouped[-1][0] == author
            and ts is not None
            and grouped[-1][2] is not None
            and (ts - grouped[-1][2]) < timedelta(hours=1)
        ):
            prev_author, prev_msg, prev_ts = grouped[-1]
            grouped[-1] = (author, f"{prev_msg} {MSG_SEP} {msg}", ts)
        else:
            grouped.append((author, msg, ts))
    return grouped


def process_whatsapp_chat_phi3(
    filepath: str,
    target_author: str,
    tokenizer,
    k_history: int = 4,
    time_gap: timedelta = timedelta(hours=3),
    system_prompt: str = "You are a helpful assistant.",
):
    """
    Returns a list of dicts with a single 'text' field formatted via Phi-3 chat template.
    Each example trains the assistant to respond as `target_author`.
    """
    print("Procesando chat (k-turns con roles) para Phi-3...")

    messages = []
    with open(filepath, "r", encoding="utf-8") as f:
        for line in f:
            ts = parse_datetime(line)
            match = re.match(r"\d+/\d+/\d+[, ]\s?\d+:\d+\s-\s([^:]+):\s(.+)", line)
            if not match:
                continue

            author = match.group(1).strip()
            raw_msg = match.group(2)
            msg = clean_text(raw_msg)

            if msg and not contains_irrelevant_data(msg):
                messages.append((author, msg, ts))

    if not messages:
        print("No se encontraron mensajes válidos.")
        return []

    messages = group_consecutive_messages(messages)
    print(f"Total de turnos agrupados: {len(messages)}")

    examples = []

    for i in range(1, len(messages)):
        author_i, msg_i, ts_i = messages[i]

        if author_i != target_author:
            continue

        context = []
        last_ts = ts_i

        for j in range(i - 1, -1, -1):
            a_j, m_j, ts_j = messages[j]

            if ts_j is not None and last_ts is not None and (last_ts - ts_j) > time_gap:
                break

            context.insert(0, (a_j, m_j))
            if ts_j is not None:
                last_ts = ts_j

            if len(context) >= k_history:
                break

        if not context:
            continue

        def fmt_turn(author: str, msg: str) -> str:
            speaker = f"[{target_author}]" if author == target_author else "[OTRO]"
            return f"{speaker} {msg}"

        context_str = " ".join(fmt_turn(a, m) for (a, m) in context)

        chat = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": context_str},
            {"role": "assistant", "content": msg_i},
        ]

        text = tokenizer.apply_chat_template(
            chat,
            tokenize=False,
            add_generation_prompt=False,
        )

        examples.append({"text": text})

    print(f"Total de ejemplos generados: {len(examples)}")
    return examples

## Tokenizer del modelo

Cargamos el tokenizer asociado al modelo base. Se utiliza durante el preprocesamiento para convertir el historial de conversación al formato exacto requerido por el chat template del modelo.

In [ ]:
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    trust_remote_code=True,
    use_fast=True,
)

# Ensure a valid padding token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("✅ Tokenizer cargado")
print("pad_token:", tokenizer.pad_token, "| eos_token:", tokenizer.eos_token)

## Procesamiento interactivo del chat

Esta celda ofrece una interfaz interactiva para cargar el archivo del chat, ingresar el nombre del autor y ajustar parámetros como el historial considerado y el tiempo máximo entre mensajes. Permite procesar el chat sin modificar código.

In [ ]:
# File upload widget for WhatsApp chat export (.txt)
chat_uploader = widgets.FileUpload(
    accept=".txt",
    multiple=False,
    description="Subir chat (.txt)"
)

# Text input for the target author (exactly as appears in the export)
author_input = widgets.Text(
    description="Autor:",
    placeholder="Nombre exactamente como figura en el chat",
    layout=widgets.Layout(width="60%")
)

# System prompt template (author name is injected at runtime)
SYSTEM_PROMPT_TEMPLATE = (
    "Sos un bot que responde mensajes de WhatsApp "
    "imitando el estilo conversacional de {author}"
)

# Slider for the number of history turns
k_history_slider = widgets.IntSlider(
    value=4,
    min=1,
    max=10,
    step=1,
    description="k_history:",
    continuous_update=False
)

# Slider for the time gap (in hours) to cut sessions
time_gap_slider = widgets.IntSlider(
    value=3,
    min=1,
    max=24,
    step=1,
    description="time_gap (h):",
    continuous_update=False
)

# Button to trigger processing
process_button = widgets.Button(
    description="Procesar chat",
    button_style="primary"
)

output_proc = widgets.Output()


def on_process_clicked(_):
    with output_proc:
        clear_output()

        # Basic validation
        if len(chat_uploader.value) == 0:
            print("Por favor, sube un archivo de chat en formato .txt.")
            return

        target_author = author_input.value.strip()
        if not target_author:
            print("Por favor, ingresa el nombre del autor exactamente como aparece en el chat.")
            return

        system_prompt = SYSTEM_PROMPT_TEMPLATE.format(author=target_author)

        # Extract uploaded file content
        upload_info = chat_uploader.value[0]
        content = bytes(upload_info["content"]).decode("utf-8-sig", errors="replace")

        # Save to a temporary file for reuse
        tmp_path = "uploaded_chat.txt"
        with open(tmp_path, "w", encoding="utf-8") as f:
            f.write(content)

        # Process chat and expose results as a global variable
        global train_examples
        train_examples = process_whatsapp_chat_phi3(
            filepath=tmp_path,
            target_author=target_author,
            tokenizer=tokenizer,
            k_history=k_history_slider.value,
            time_gap=timedelta(hours=time_gap_slider.value),
            system_prompt=system_prompt,
        )

        print(f"Ejemplos generados: {len(train_examples)}")
        if train_examples:
            print("\nEjemplo de TEXT (Phi-3 chat template):")
            ex = train_examples[0]["text"]
            print(ex[:800] + ("..." if len(ex) > 800 else ""))

        print("\nVariable disponible para las siguientes celdas: 'train_examples'.")


process_button.on_click(on_process_clicked)

display(
    widgets.VBox([
        widgets.HTML("<b>Procesamiento interactivo del chat de WhatsApp</b>"),
        chat_uploader,
        author_input,
        k_history_slider,
        time_gap_slider,
        process_button,
        output_proc,
    ])
)

## Creación del dataset, limpieza y guardado

Esta celda toma los ejemplos generados previamente, construye un DataFrame, aplica una limpieza básica (elimina ejemplos triviales y enlaces) y guarda un archivo CSV con el dataset inicial. Además, muestra algunos ejemplos aleatorios para inspeccionar el resultado.

In [ ]:
# Safety check: train_examples must exist
if "train_examples" not in globals():
    raise RuntimeError(
        "La variable 'train_examples' no existe. "
        "Ejecuta primero el procesamiento interactivo del chat (Phi-3) para generar 'train_examples'."
    )


def build_dataset_from_text(
    examples,
    min_words: int = 12,
    filter_links: bool = True,
):
    """
    Build a DataFrame from Phi-3 formatted samples (each sample is {"text": ...}).
    The function drops invalid rows, filters very short samples, and optionally removes URLs.
    """
    df = pd.DataFrame(examples)
    if "text" not in df.columns:
        raise ValueError("Cada ejemplo debe tener la clave 'text'.")

    df = df.dropna(subset=["text"]).reset_index(drop=True)
    df = df[df["text"].str.split().str.len() >= min_words].reset_index(drop=True)

    if filter_links:
        df = df[~df["text"].str.contains(r"http|www|\.com", regex=True)].reset_index(drop=True)

    return df


# Default configuration
min_words_default = 12
filter_links_default = True
output_path_default = "train_data_phi3_text.csv"

data = build_dataset_from_text(
    train_examples,
    min_words=min_words_default,
    filter_links=filter_links_default,
)

data.to_csv(output_path_default, index=False, quoting=csv.QUOTE_ALL)

print(f"Dataset inicial guardado → {len(data)} ejemplos.")
print("\nVista aleatoria de algunos ejemplos:\n")

for _ in range(min(5, len(data))):
    s = data.sample(1).iloc[0]
    text = s["text"]
    print(f"TEXT (Phi-3):\n{text[:900]}{'...' if len(text) > 900 else ''}\n{'-'*70}")


## Filtrado semántico global

Esta celda aplica un filtrado semántico basado en embeddings para conservar únicamente ejemplos con alta coherencia semántica entre el mensaje del usuario y la respuesta del asistente.
Se calcula la similitud coseno, se filtran ejemplos ruidosos y se guarda el dataset final.

In [ ]:
# Load the sentence transformer model once
if "model_emb" not in globals():
    model_emb = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")


def extract_user_and_assistant(phi3_text: str):
    """
    Extracts the first <|user|> ... <|end|> and <|assistant|> ... <|end|> blocks
    from a Phi-3 formatted text.
    Returns (user_text, assistant_text) or (None, None) if parsing fails.
    """
    if not isinstance(phi3_text, str):
        return None, None

    user_match = re.search(r"<\|user\|>\s*(.*?)\s*<\|end\|>", phi3_text, flags=re.DOTALL)
    asst_match = re.search(r"<\|assistant\|>\s*(.*?)\s*<\|end\|>", phi3_text, flags=re.DOTALL)

    user_text = user_match.group(1).strip() if user_match else None
    assistant_text = asst_match.group(1).strip() if asst_match else None
    return user_text, assistant_text


# Build user / assistant columns from Phi-3 text
ua = data["text"].apply(extract_user_and_assistant)
data["user_text"] = ua.apply(lambda x: x[0])
data["assistant_text"] = ua.apply(lambda x: x[1])

# Drop rows that could not be parsed correctly
data = data.dropna(subset=["user_text", "assistant_text"]).reset_index(drop=True)

# Encode user and assistant messages
emb_users = model_emb.encode(
    data["user_text"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True,
)
emb_assts = model_emb.encode(
    data["assistant_text"].tolist(),
    convert_to_tensor=True,
    show_progress_bar=True,
)

# Compute cosine similarity for each pair
similarities = util.cos_sim(emb_users, emb_assts).diagonal().cpu().numpy()
data["similarity"] = similarities

print(f"Media de similitud: {data['similarity'].mean():.3f}")

# Filtering configuration
SIM_THRESHOLD = 0.30
MAX_USER_LEN = 1200
MAX_ASSISTANT_LEN = 800

filtered = data[
    (data["similarity"] > SIM_THRESHOLD)
    & (data["user_text"].str.len() < MAX_USER_LEN)
    & (data["assistant_text"].str.len() < MAX_ASSISTANT_LEN)
].reset_index(drop=True)

# Save final training dataset (Phi-3 text only)
output_filtered_path = "filtered_train_phi3_text.csv"
filtered[["text"]].to_csv(output_filtered_path, index=False, quoting=csv.QUOTE_ALL)

# Optional debug dataset for inspection
output_debug_path = "filtered_train_phi3_debug.csv"
filtered[["user_text", "assistant_text", "similarity"]].to_csv(
    output_debug_path, index=False, quoting=csv.QUOTE_ALL
)

print(f"Dataset final (solo text) guardado en '{output_filtered_path}'.")
print(f"Dataset debug guardado en '{output_debug_path}'.")
print(f"Total de ejemplos útiles: {len(filtered)} (de {len(data)}).")

print("\nVista rápida de algunos ejemplos filtrados:\n")
for _ in range(min(3, len(filtered))):
    s = filtered.sample(1).iloc[0]
    print(
        f"USER:\n{s['user_text']}\n"
        f"→ ASSISTANT:\n{s['assistant_text']}\n"
        f"Similarity: {s['similarity']:.3f}\n"
        + "-" * 70
    )

## Preparación del dataset para entrenamiento

Esta celda carga el dataset filtrado en formato Phi-3 (text), realiza una limpieza mínima, lo convierte a datasets.Dataset y lo divide en entrenamiento y validación.

In [ ]:
# Load and minimally clean the dataset (Phi-3 format: 'text' column)
data = pd.read_csv("filtered_train_phi3_text.csv")
data = data.dropna(subset=["text"])
data = data[data["text"].str.strip() != ""].reset_index(drop=True)
data.to_csv("filtered_train_phi3_text.csv", index=False, quoting=csv.QUOTE_ALL)

# Load into HuggingFace datasets
dataset = load_dataset("csv", data_files="filtered_train_phi3_text.csv")

# Train/validation split
train_test = dataset["train"].train_test_split(test_size=0.2, seed=42)
datasets = DatasetDict(
    {"train": train_test["train"], "validation": train_test["test"]}
)

print(datasets)
print("Ejemplo:")
print(datasets["train"][0]["text"][:400])

## Carga del modelo base Phi-3 con QLoRA

Esta celda carga el modelo base Phi-3 Mini Instruct utilizando cuantización en 4 bits (QLoRA).
Se inicializa el tokenizer, se configura el padding y se prepara el modelo para fine-tuning eficiente mediante adapters LoRA.

In [ ]:
# QLoRA configuration (4-bit quantization)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Base model loaded in 4-bit mode
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,  # Disabled for training with gradient checkpointing
)

# Prepare model for k-bit (QLoRA) training
model = prepare_model_for_kbit_training(model)

print("✅ Modelo base cargados en 4-bit (listos para aplicar LoRA).")

## Aplicación de adapters LoRA al modelo base

Esta celda configura y aplica LoRA (Low-Rank Adaptation) sobre el modelo base Phi-3 previamente cargado.
Se definen los hiperparámetros de LoRA y los módulos objetivo que serán ajustados durante el fine-tuning.

In [ ]:
# LoRA configuration
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        # Common projection layers used in transformer architectures
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
)

# Attach LoRA adapters to the base model
model = get_peft_model(model, lora_config)

# Display number of trainable vs frozen parameters
model.print_trainable_parameters()

print("✅ LoRA aplicado (modelo listo para SFTTrainer).")

### Análisis estadístico y calidad del dataset final (Phi-3)

Este bloque analiza exclusivamente el dataset conversacional final utilizado para el entrenamiento del modelo Phi-3 Instruct.
El objetivo es evaluar la calidad del conjunto de datos que efectivamente se usa durante el fine-tuning supervisado, verificando su tamaño, diversidad lingüística y adecuación al contexto del modelo.

En particular, se reportan las siguientes métricas:

* Samples: cantidad total de ejemplos de entrenamiento presentes en el dataset final.

* Exact duplicates: número de ejemplos idénticos detectados. Un valor bajo indica que no hay sobre-representación artificial de conversaciones repetidas.

* Chars avg: longitud promedio de los ejemplos medida en caracteres. Proporciona una noción general del tamaño del texto sin depender del tokenizer.

* Words avg: cantidad promedio de palabras por ejemplo, utilizada como medida lingüística intuitiva de la longitud del contenido.

* Words p95: percentil 95 de la longitud en palabras. Indica que el 95% de los ejemplos tiene una longitud menor o igual a este valor y permite identificar conversaciones largas.

* Unique words: cantidad total de palabras distintas presentes en el dataset final, lo que refleja el tamaño efectivo del vocabulario.

* Vocab richness (%): proporción entre palabras únicas y el total de palabras. Valores relativamente bajos son esperables en chats personales y reflejan consistencia de estilo y repetición de expresiones.

* Tokens avg: cantidad promedio de tokens por ejemplo según el tokenizer de Phi-3, que representa lo que realmente procesa el modelo durante el entrenamiento.

* Tokens p95: percentil 95 de la longitud en tokens. Es una métrica clave para estimar el uso del contexto y evitar truncamiento durante el fine-tuning.

Además, se incluye un ranking de las palabras más frecuentes del dataset final, presentado en dos vistas complementarias:

* Con muletillas: frecuencia cruda de palabras, útil para detectar expresiones repetidas, muletillas y patrones conversacionales característicos del autor.

* Sin muletillas: frecuencia filtrada eliminando stopwords comunes y ruido residual del template, útil para identificar temas, contenidos y vocabulario informativo dominante.

Finalmente, se muestra una nube de palabras construida a partir del vocabulario informativo del dataset final, donde las palabras más frecuentes aparecen visualmente destacadas. Esta visualización permite caracterizar de forma rápida el estilo y los temas predominantes que el modelo aprenderá durante el entrenamiento.


In [ ]:
try:
    from wordcloud import WordCloud
    WORDCLOUD_AVAILABLE = True
except Exception:
    WORDCLOUD_AVAILABLE = False

# Safety check: final dataset only
assert "filtered" in globals() and isinstance(filtered, pd.DataFrame) and "text" in filtered.columns, \
    "❌ 'filtered' no existe o no tiene columna 'text' (dataset final de Phi-3)"

texts = filtered["text"].astype(str).tolist()
texts = [t for t in texts if t.strip()]

# Regex & helpers
WORD_RE = re.compile(r"[a-záéíóúñü0-9']+")

# Phi-3 template tags
TAG_RE_GENERIC = re.compile(r"<\|.*?\|>")

# Remove full system block (<|system|> ... <|end|>)
SYSTEM_BLOCK_PHI3_RE = re.compile(
    r"<\|system\|>\s*.*?\s*<\|end\|>",
    re.DOTALL | re.IGNORECASE
)

# Dynamic author filtering
AUTHOR_STOP_WORDS = set()
if "author_input" in globals() and author_input.value.strip():
    AUTHOR_STOP_WORDS = {
        w for w in re.findall(r"[a-záéíóúñü]+", author_input.value.strip().lower())
        if len(w) >= 2
    }

# Stopwords
STOP_WORDS_EXTRA = {
    "user", "assistant", "system",
    "metadata", "knowledge", "cutoff", "date", "today",
    "reasoning", "mode", "custom", "instructions",
    "im_start", "im_end", "end", "msg_sep",
    "think", "no_think"
}.union(AUTHOR_STOP_WORDS)

SPANISH_STOPWORDS = {
    "el","la","los","las","un","una","unos","unas",
    "yo","me","te","se","lo","le","nos","les",
    "de","que","y","o","pero","si","no","es","en","con","por","para",
    "ya","bien","eh","ah","oh","xd","jaja","jajaja"
}

# Normalization
def normalize_for_stats(text: str) -> str:
    if not isinstance(text, str):
        return ""

    # Remove full system prompt
    text = SYSTEM_BLOCK_PHI3_RE.sub(" ", text)

    # Remove remaining tags
    text = TAG_RE_GENERIC.sub(" ", text)

    # Normalize
    text = text.replace("\n", " ").lower()
    text = re.sub(r"\d+", " ", text)
    text = re.sub(r"\s+", " ", text).strip()
    return text

def extract_words(text: str):
    words = WORD_RE.findall(normalize_for_stats(text))
    return [w for w in words if w not in STOP_WORDS_EXTRA and len(w) >= 2]

def extract_informative_words(text: str):
    return [w for w in extract_words(text) if w not in SPANISH_STOPWORDS]

# Dataset statistics

char_lengths = [len(t) for t in texts]
word_lengths = [len(extract_words(t)) for t in texts]

vocab_raw = Counter()
for t in texts:
    vocab_raw.update(extract_words(t))

stats = {
    "samples": len(texts),
    "exact_duplicates": int(pd.Series(texts).duplicated().sum()),
    "chars_avg": float(sum(char_lengths) / max(1, len(texts))),
    "words_avg": float(sum(word_lengths) / max(1, len(texts))),
    "words_p95": float(pd.Series(word_lengths).quantile(0.95)),
    "unique_words": int(len(vocab_raw)),
    "vocab_richness_pct": 100.0 * len(vocab_raw) / max(1, sum(vocab_raw.values())),
}

# Token stats (if tokenizer exists)
if "tokenizer" in globals() and tokenizer is not None:
    try:
        token_lengths = [
            len(tokenizer(t, add_special_tokens=False).input_ids)
            for t in texts
        ]
        stats.update({
            "tokens_avg": float(sum(token_lengths) / max(1, len(token_lengths))),
            "tokens_p95": float(pd.Series(token_lengths).quantile(0.95)),
        })
    except Exception:
        pass

summary_df = pd.DataFrame([stats])

print("\n📊 Resumen estadístico — Dataset final (Phi-3)")
display(summary_df)


# Top words: with / without stopwords
TOP_K = 20
print(f"\n🔝 Top {TOP_K} palabras (dataset final)")

top_raw = pd.DataFrame(
    vocab_raw.most_common(TOP_K),
    columns=["palabra", "frecuencia"]
)

vocab_info = Counter()
for t in texts:
    vocab_info.update(extract_informative_words(t))

top_info = pd.DataFrame(
    vocab_info.most_common(TOP_K),
    columns=["palabra", "frecuencia"]
)

max_len = max(len(top_raw), len(top_info))
top_raw = top_raw.reindex(range(max_len))
top_info = top_info.reindex(range(max_len))

spacer = pd.DataFrame({"": [""] * max_len})
spacer.columns = pd.MultiIndex.from_product([[""], [""]])

top_raw.columns = pd.MultiIndex.from_product([["Con muletillas"], top_raw.columns])
top_info.columns = pd.MultiIndex.from_product([["Sin muletillas"], top_info.columns])

display(pd.concat([top_raw, spacer, top_info], axis=1))

# Word cloud (informative words)
if WORDCLOUD_AVAILABLE:
    print("\n🖼️ Nube de palabras — Dataset final (sin stopwords)")
    wc_vocab = Counter()
    for t in texts:
        wc_vocab.update(extract_informative_words(t))

    wc = WordCloud(
        width=1200,
        height=600,
        background_color="white",
        max_words=120,
        collocations=False
    ).generate_from_frequencies(dict(wc_vocab))

    plt.figure(figsize=(12, 6))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.show()
else:
    print("\nℹ️ WordCloud no disponible (pip install wordcloud)")

# Final info
print("• Modelo objetivo: Phi-3 Instruct")
print("• Dataset analizado: dataset final de entrenamiento (columna 'text')")
print("• Nota: el bloque <|system|>…<|end|> se elimina SOLO para estadísticas")
print(f"• Total de ejemplos: {len(texts)}")

## Argumentos de entrenamiento

Define los hiperparámetros principales de entrenamiento para el fine-tuning del modelo (número de épocas, batch size, tasa de aprendizaje, etc.).

In [ ]:
training_args = SFTConfig(
    output_dir = f"./models/{author_input.value.strip()}_whatsapp_phi3",
    num_train_epochs=3,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,

    learning_rate=1e-4,  # Typical LoRA/QLoRA learning rate range
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    logging_steps=10,
    save_strategy="steps",
    save_steps=200,
    save_total_limit=2,

    eval_strategy="no",
    report_to="none",

    bf16=True,
    fp16=False,
    gradient_checkpointing=True,

    dataset_text_field="text",
    max_length=2048,
    packing=True,
)

## Configuración del Trainer

Configura el objeto que orquesta el entrenamiento con TRL (SFTTrainer), conectando el modelo, los datasets y los argumentos. La tokenización y el armado de labels se gestionan internamente a partir del campo text.

In [ ]:
trainer = None

try:
    # Newer TRL versions
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=datasets["train"],
        eval_dataset=datasets["validation"],
        processing_class=tokenizer,
    )
    print("✅ SFTTrainer creado usando processing_class=tokenizer")
except TypeError:
    # Older TRL versions
    trainer = SFTTrainer(
        model=model,
        args=training_args,
        train_dataset=datasets["train"],
        eval_dataset=datasets["validation"],
        tokenizer=tokenizer,
    )
    print("✅ SFTTrainer creado usando tokenizer=tokenizer")

## Entrenamiento y guardado del modelo

Esta celda entrena el modelo y guarda los adapters LoRA y el tokenizer en un directorio cuyo nombre se genera automáticamente en función del autor seleccionado en la etapa de procesamiento interactivo.

In [ ]:
# Build dynamic save directory from author_input
if "author_input" in globals():
    author_name = author_input.value.strip()
    safe_author = re.sub(r"[^A-Za-z0-9_-]", "_", author_name)
    save_dir = f"./bot_{safe_author}" if safe_author else "./bot_model"
else:
    save_dir = "./bot_model"

Path(save_dir).mkdir(parents=True, exist_ok=True)

print(f"Guardando adapters LoRA + tokenizer en: {save_dir}")

try:
    trainer.train()
finally:
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    gc.collect()

# Save LoRA adapters + tokenizer (QLoRA workflow)
trainer.model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

print(f"✅ Adapters LoRA + tokenizer guardados en: {save_dir}")

## Comparador de respuestas entre modelos

Esta celda carga el modelo base y el modelo fine-tuneado, y define un comparador para generar respuestas con ambos a partir de un mismo prompt. De esta forma se puede inspeccionar cualitativamente el efecto del fine-tuning.

In [ ]:
# Ensure save_dir exists (from training step)
if "save_dir" not in globals():
    save_dir = "./bot_model"

# Ensure a system prompt exists
if "SYSTEM_PROMPT" not in globals() or not isinstance(SYSTEM_PROMPT, str) or not SYSTEM_PROMPT.strip():
    SYSTEM_PROMPT = f"Sos un bot que responde mensajes de WhatsApp imitando el estilo conversacional de ${author_input.value.strip()}"

# Load base Phi-3 model (4-bit)
base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)
base_model.eval()

# Load a separate base model instance for the LoRA-augmented model (clean comparison)
fine_base_model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    use_cache=False,
)
fine_model = PeftModel.from_pretrained(fine_base_model, save_dir)
fine_model.eval()

print(f"✅ Base model: {MODEL_ID}")
print(f"✅ Fine model (LoRA adapters): {save_dir}")


def generate_response_phi3(
    user_prompt: str,
    model,
    tokenizer,
    system_prompt: str = SYSTEM_PROMPT,
    max_new_tokens: int = 80,
    temperature: float = 0.8,
    top_p: float = 0.92,
) -> str:
    """Generates a single assistant response using Phi-3 chat template."""
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt.strip()},
    ]

    prompt_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt_text, return_tensors="pt")

    # Move tensors to the first parameter device when possible (works with device_map="auto")
    try:
        param_device = next(model.parameters()).device
        inputs = {k: v.to(param_device) for k, v in inputs.items()}
    except StopIteration:
        pass

    # Disable KV cache to avoid DynamicCache/seen_tokens incompatibilities in some environments
    if hasattr(model, "config"):
        model.config.use_cache = False
    if hasattr(model, "generation_config"):
        model.generation_config.use_cache = False

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=True,
            temperature=temperature,
            top_p=top_p,
            repetition_penalty=1.15,
            no_repeat_ngram_size=3,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            use_cache=False,
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Keep only the assistant segment
    if "<|assistant|>" in decoded:
        decoded = decoded.split("<|assistant|>")[-1]
    if "<|end|>" in decoded:
        decoded = decoded.split("<|end|>")[0]

    return re.sub(r"\s+", " ", decoded).strip() or "(no generated response)"


def compare_models(user_prompt: str):
    print(f"\nPrompt:\n{user_prompt}")

    base_resp = generate_response_phi3(user_prompt, base_model, tokenizer, system_prompt=SYSTEM_PROMPT)
    fine_resp = generate_response_phi3(user_prompt, fine_model, tokenizer, system_prompt=SYSTEM_PROMPT)

    print("\n----------------------------------")
    print("Base model:\n", base_resp)
    print("----------------------------------")
    print("Fine-tuned (LoRA):\n", fine_resp)
    print("----------------------------------")


def extract_user_from_text(phi3_text: str):
    """Extracts the user message from a Phi-3 formatted sample."""
    m = re.search(r"<\|user\|>\s*(.*?)\s*<\|end\|>", phi3_text, flags=re.DOTALL)
    return m.group(1).strip() if m else None


val_user_prompts = []
for ex in datasets["validation"]:
    u = extract_user_from_text(ex["text"])
    if u:
        val_user_prompts.append(u)

sample_prompts = random.sample(val_user_prompts, min(5, len(val_user_prompts)))

print("\n=== Comparación de modelos usando muestras de validación ===")
for p in sample_prompts:
    compare_models(p)

## Evaluación cuantitativa y reporte final

Esta celda evalúa cuantitativamente el modelo base y el modelo fine-tuneado sobre el conjunto de validación. Se calcula la pérdida media de cross-entropy y la perplexity aproximada, y se reporta la mejora relativa del modelo fine-tuneado. Finalmente, se muestra una comparación gráfica sencilla entre ambos modelos.

In [ ]:
sns.set_theme(style="whitegrid")

def evaluate_model_phi3(
    model,
    dataset,
    tokenizer,
    max_examples: int = 200,
    max_length: int = 2048,
):
    """
    Evaluates a causal LM on a dataset with a 'text' column (Phi-3 chat template).
    Computes mean cross-entropy loss and perplexity (exp(loss)) over the selected subset.

    Note: This is a global metric over the full sample (system + user + assistant).
    """
    model.eval()
    losses = []

    with torch.no_grad():
        for i, example in enumerate(tqdm(dataset, desc="Evaluando...")):
            if i >= max_examples:
                break

            text = example.get("text", None)
            if not isinstance(text, str) or not text.strip():
                continue

            enc = tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                max_length=max_length,
            )

            # Move tensors to a valid parameter device (works with device_map="auto")
            try:
                param_device = next(model.parameters()).device
                enc = {k: v.to(param_device) for k, v in enc.items()}
            except StopIteration:
                pass

            input_ids = enc["input_ids"]
            outputs = model(input_ids=input_ids, labels=input_ids)
            losses.append(float(outputs.loss))

    n = len(losses)
    avg_loss = sum(losses) / max(1, n)

    try:
        perp = math.exp(avg_loss)
    except OverflowError:
        perp = float("inf")

    return {"cross_entropy": avg_loss, "perplexity": perp, "n": n}


metrics_base = evaluate_model_phi3(
    base_model, datasets["validation"], tokenizer, max_examples=200, max_length=2048
)
metrics_fine = evaluate_model_phi3(
    fine_model, datasets["validation"], tokenizer, max_examples=200, max_length=2048
)

print("\nResultados de evaluación:")
print(
    f"Base model  → CE: {metrics_base['cross_entropy']:.3f} | "
    f"PPL: {metrics_base['perplexity']:.2f} | n={metrics_base['n']}"
)
print(
    f"Fine-tuned  → CE: {metrics_fine['cross_entropy']:.3f} | "
    f"PPL: {metrics_fine['perplexity']:.2f} | n={metrics_fine['n']}"
)

if metrics_base["perplexity"] != float("inf") and metrics_base["perplexity"] > 0:
    improvement = (
        (metrics_base["perplexity"] - metrics_fine["perplexity"])
        / metrics_base["perplexity"]
        * 100
    )
    print(f"Mejora relativa en perplexity: {improvement:.2f}%")
else:
    print("No se pudo calcular mejora relativa (perplexity base inválida).")

# Simple barplot comparison
plt.figure(figsize=(8, 5))
barplot = sns.barplot(
    x=["Base (Phi-3)", "Fine-tuned"],
    y=[metrics_base["perplexity"], metrics_fine["perplexity"]],
    palette=["#95a5a6", "#2ecc71"]
)

# Annotate bar values
for p in barplot.patches:
    barplot.annotate(
        f"{p.get_height():.1f}",
        (p.get_x() + p.get_width() / 2.0, p.get_height()),
        ha="center",
        va="center",
        xytext=(0, 9),
        textcoords="offset points",
    )
plt.ylim(0, 2000)

plt.title("Comparación de perplexity")
plt.ylabel("Perplexity")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()